<a href="https://colab.research.google.com/github/mizutokage/Kaggle/blob/main/Titanic/xgb_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold

import xgboost as xgb

In [143]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train = train[['PassengerId', 'Survived', 'Pclass',
         'Age', 'Sex', 'SibSp', 
         'Parch', 'Fare', 'Embarked']]

test = test[['PassengerId', 'Pclass',
         'Age', 'Sex', 'SibSp',
         'Parch', 'Fare', 'Embarked']]

In [144]:
train = train.fillna(train['Age'].mean())
train['Sex'][train['Sex'] == 'male'] = 0
train['Sex'][train['Sex'] == 'female'] = 1
train['Embarked'][train['Embarked'] == 'C'] = 0
train['Embarked'][train['Embarked'] == 'S'] = 1
train['Embarked'][train['Embarked'] == 'Q'] = 2

test['Sex'][test['Sex'] == 'male'] = 0
test['Sex'][test['Sex'] == 'female'] = 1
test['Embarked'][test['Embarked'] == 'C'] = 0
test['Embarked'][test['Embarked'] == 'S'] = 1
test['Embarked'][test['Embarked'] == 'Q'] = 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [145]:
train.head()

,PassengerId,Survived,Pclass,Age,Sex,SibSp,Parch,Fare,Embarked
0,1,0,3,22.0,0,1,0,7.2500,1
1,2,1,1,38.0,1,1,0,71.2833,0
2,3,1,3,26.0,1,0,0,7.9250,1
3,4,1,1,35.0,1,1,0,53.1000,1
4,5,0,3,35.0,0,0,0,8.0500,1


In [146]:
test.head()

,PassengerId,Pclass,Age,Sex,SibSp,Parch,Fare,Embarked
0,892,3,34.5,0,0,0,7.8292,2
1,893,3,47.0,1,1,0,7.0000,1
2,894,2,62.0,0,0,0,9.6875,2
3,895,3,27.0,0,0,0,8.6625,1
4,896,3,22.0,1,1,1,12.2875,1


In [147]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
train['Sex'] = lbl.fit_transform(train['Sex'].astype(int))
train['Embarked'] = lbl.fit_transform(train['Embarked'].astype(int))
test['Sex'] = lbl.fit_transform(test['Sex'].astype(int))
test['Embarked'] = lbl.fit_transform(test['Embarked'].astype(int))

In [148]:
train.head()

,PassengerId,Survived,Pclass,Age,Sex,SibSp,Parch,Fare,Embarked
0,1,0,3,22.0,0,1,0,7.2500,1
1,2,1,1,38.0,1,1,0,71.2833,0
2,3,1,3,26.0,1,0,0,7.9250,1
3,4,1,1,35.0,1,1,0,53.1000,1
4,5,0,3,35.0,0,0,0,8.0500,1


In [149]:
test.head()

,PassengerId,Pclass,Age,Sex,SibSp,Parch,Fare,Embarked
0,892,3,34.5,0,0,0,7.8292,2
1,893,3,47.0,1,1,0,7.0000,1
2,894,2,62.0,0,0,0,9.6875,2
3,895,3,27.0,0,0,0,8.6625,1
4,896,3,22.0,1,1,1,12.2875,1


In [150]:
seed = 88
X = train.drop(['Survived'], axis=1)
y = train['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=seed)
X

,PassengerId,Pclass,Age,Sex,SibSp,Parch,Fare,Embarked
0,1,3,22.000000,0,1,0,7.2500,1
1,2,1,38.000000,1,1,0,71.2833,0
2,3,3,26.000000,1,0,0,7.9250,1
3,4,1,35.000000,1,1,0,53.1000,1
4,5,3,35.000000,0,0,0,8.0500,1
...,...,...,...,...,...,...,...,...
886,887,2,27.000000,0,0,0,13.0000,1
887,888,1,19.000000,1,0,0,30.0000,1
888,889,3,29.699118,1,1,2,23.4500,1
889,890,1,26.000000,0,0,0,30.0000,0


In [151]:
from bayes_opt import BayesianOptimization

In [152]:
def xgb_evaluate(min_child_weight, subsample, colsample_bytree, max_depth):
      params = {'metric': 'error',
              'objective':'binary:logistic',
              'n_estimators':50000,
              'random_state':42,
              'boosting_type':'gbdt',
              'learning_rate':0.01,              
              'min_child_weight': int(min_child_weight),
              'max_depth': int(max_depth),
              'colsample_bytree': colsample_bytree,
              'subsample': subsample,
             }

      cls = xgb.XGBClassifier()
      cls.set_params(**params)
      cls.fit(X_train,
              y_train,
              early_stopping_rounds=50,
              eval_set=[(X_test, y_test)],
              eval_metric='error',
              verbose=0)
      
      pred = cls.predict(X_test)
      score = accuracy_score(y_test, pred)
      return score

In [153]:
xgb_bo = BayesianOptimization(xgb_evaluate,
                              {'min_child_weight': (1, 20),
                               'subsample': (.1, 1),
                               'colsample_bytree': (.1, 1),
                               'max_depth': (1, 50)},
                               random_state=10)

In [154]:
xgb_bo.maximize(init_points=15, n_iter=50, acq='ei')

|   iter    |  target   | colsam... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------
|  1        |  0.7724   |  0.7942   |  2.017    |  13.04    |  0.7739   |
|  2        |  0.7985   |  0.5487   |  12.02    |  4.763    |  0.7845   |
|  3        |  0.7873   |  0.2522   |  5.329    |  14.02    |  0.9581   |
|  4        |  0.7687   |  0.1036   |  26.1     |  16.44    |  0.6513   |
|  5        |  0.7724   |  0.7496   |  15.3     |  18.44    |  0.7431   |
|  6        |  0.7836   |  0.5883   |  7.966    |  8.093    |  0.7067   |
|  7        |  0.7761   |  0.4976   |  22.27    |  12.74    |  0.5618   |
|  8        |  0.7724   |  0.6854   |  30.45    |  16.3     |  0.5695   |
|  9        |  0.7985   |  0.9178   |  16.64    |  2.719    |  0.3706   |
|  10       |  0.7575   |  0.2026   |  41.61    |  1.891    |  0.6637   |
|  11       |  0.7985   |  0.5928   |  41.15    |  4.78     |  0.8712   |
|  12       |  0.8097   |  0.4165   | 

In [157]:
optimized_params = xgb_bo.max['params']
optimized_params['max_depth'] = int(optimized_params['max_depth'])
optimized_params

{'colsample_bytree': 0.8581395042489772,
 'max_depth': 19,
 'min_child_weight': 1.8082255658216937,
 'subsample': 0.9943584053999677}

In [158]:
fixed_params = {'metric':'error',
                'objective':'binary:logistic',
                'n_estimators':50000,
                'random_state':seed,
                'booster':'gbtree',
                'learning_rate':0.01}

In [159]:
cls = xgb.XGBClassifier()
cls.set_params(**fixed_params, **optimized_params)
cls.fit(X_train,
        y_train,
        early_stopping_rounds=50,
        eval_set=[(X_test, y_test)],
        eval_metric='error',
        verbose=0)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8581395042489772, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=19,
              metric='error', min_child_weight=1.8082255658216937, missing=None,
              n_estimators=50000, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=88, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=0.9943584053999677, verbosity=1)

In [160]:
pred = cls.predict(test)
PassengerId = np.array(test['PassengerId']).astype(int)

sol = pd.DataFrame(pred, PassengerId, columns=['Survived'])
sol.to_csv("my_pred.csv", index_label=["PassengerId"])